In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import pandas as pd
from pyDOE import lhs

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [3]:
# Building ANN
class Net(nn.Module):

    def __init__(self, n_layers=20, n_nodes=64):
        super().__init__()
        self.fcin = nn.Linear(2, n_nodes)
        self.hidden = nn.ModuleList([nn.Linear(n_nodes,n_nodes) for i in range(n_layers)])
        self.fcout = nn.Linear(n_nodes, 3)

    def forward(self, x):
        x = torch.sin(self.fcin(x))
        for layers in self.hidden:
            x = torch.sin(layers(x))
        x = self.fcout(x)
        
        u = x[:,0].view(len(x[:,0]),1)
        v = x[:,1].view(len(x[:,1]),1)
        p = x[:,2].view(len(x[:,2]),1)

        return u, v, p

# Auto Differentiation
def auto_diff(u, v, p, coords):
    size = torch.ones(u.size()).to(device)
    
    du = grad(u, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]
    dv = grad(v, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]
    dp = grad(p, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]

    u_x = du[:,0].view(len(du[:,0]),1)
    u_y = du[:,1].view(len(du[:,1]),1)

    v_x = dv[:,0].view(len(dv[:,0]),1)
    v_y = dv[:,1].view(len(dv[:,1]),1)
    
    p_x = dp[:,0].view(len(dp[:,0]),1)
    p_y = dp[:,1].view(len(dp[:,1]),1)
    
    du_x = grad(u_x, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]
    du_y = grad(u_y, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]

    dv_x = grad(v_x, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]
    dv_y = grad(v_y, coords, grad_outputs=size, create_graph=True, allow_unused=True)[0]
    
    u_xx = du_x[:,0].view(len(du_x[:,0]),1)
    u_yy = du_y[:,1].view(len(du_y[:,1]),1)

    v_xx = dv_x[:,0].view(len(dv_x[:,0]),1)
    v_yy = dv_y[:,1].view(len(dv_y[:,1]),1)
    
    return u_x, u_y, v_x, v_y, p_x, p_y, u_xx, u_yy, v_xx, v_yy

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [4]:
# Loss Function
def loss_fn(INLET, CYLINDER, FLUID):
    inlet_velocity = 1.0
    angle_of_attack = 0
    inlet_target_u = torch.ones(len(INLET),1).to(device)
    inlet_target_v = torch.zeros(len(INLET),1).to(device)
    inlet_zero_target = torch.zeros(len(INLET),1).to(device)
    cylinder_zero_target = torch.zeros(len(CYLINDER),1).to(device)
    fluid_zero_target = torch.zeros(len(FLUID),1).to(device)
    
    MSEloss = nn.MSELoss()
    loss = 0
    # Train Inlet
    # Zero Parameters
    model.zero_grad()
    graph_inlet_loss = 0
    # Foward Propagation
    u, v, p = model(INLET)
    # Inlet Loss 
    loss_inlet = MSEloss(u, inlet_target_u) + MSEloss(v, inlet_target_v)

    # Train Cylinder
    # Zero Parameters
    model.zero_grad()
    graph_wall_loss = 0   
    # Foward Propagation
    u, v, p = model(CYLINDER)
    # Cylinder Loss
    loss_cylinder = MSEloss(u, cylinder_zero_target) + MSEloss(v, cylinder_zero_target)
    
    # Train Fluid
    # Zero Parameters
    model.zero_grad()
    graph_ns_loss = 0
    # Foward Propagation
    u, v, p = model(FLUID)
    u_x, u_y, v_x, v_y, p_x, p_y, u_xx, u_yy, v_xx, v_yy = auto_diff(u, v, p, FLUID)
    # Fluid Loss
    nu = 2e-2
    R_con = u_x + v_y  # Continuity
    R_xmom = u*u_x + v*u_y + p_x - nu*(u_xx + u_yy) # x momentum
    R_ymom = u*v_x + v*v_y + p_y - nu*(v_xx + v_yy) # y momentum     
    loss_fluid = MSEloss(R_con, fluid_zero_target) + MSEloss(R_xmom, fluid_zero_target) + MSEloss(R_ymom, fluid_zero_target)
    
    return loss_inlet, loss_cylinder, loss_fluid

def closure():
    optimizer_LBFGS.zero_grad()
    loss_inlet, loss_cylinder, loss_fluid = loss_fn(INLET, CYLINDER, FLUID)
    loss = loss_inlet + loss_cylinder + loss_fluid
    loss.backward()
    return loss

In [ ]:
### Create data points
lb = -1 #Left bound
rb = 3 #Right bound
tb = 1 #Top bound
bb = -1 #Bottom bound

fluid_points = 10000

INLET = [lb,bb] + [0.0, tb-bb] * lhs(2, 200)
TOP = [lb,tb] + [rb-lb, 0.0] * lhs(2,200)
BOTTOM = [lb,bb] + [rb-lb, 0.0] * lhs(2,200)

INLET = np.concatenate((INLET,TOP,BOTTOM), axis = 0)


theta = np.linspace(0, 360, 180)
cylinder_x = 0.05*np.cos(theta*np.pi/180)
cylinder_y = 0.05*np.sin(theta*np.pi/180)
cylinder_x = cylinder_x.flatten()[:, None]
cylinder_y = cylinder_y.flatten()[:, None]
CYLINDER = np.concatenate((cylinder_x, cylinder_y), axis = 1)

fluid = ([lb, bb] + [rb-lb, tb-bb] * lhs(2,fluid_points)).tolist()

fluid_boolean = []

for point in fluid:
    if point[0]**2 + point[1]**2 <= 0.05**2:
        fluid_boolean.append(False)
    elif point[0] == -0.2:
        fluid_boolean.append(False)
    else:
        fluid_boolean.append(True)        

FLUID = []
for i in range(len(fluid_boolean)):
    if fluid_boolean[i]:
        FLUID.append(fluid[i])
FLUID = np.array(FLUID)

# Visualise Points
fig, ax = plt.subplots(figsize = (20, 20))
plt.scatter(np.array(CYLINDER)[:,0],np.array(CYLINDER)[:,1], s = 0.5)
plt.scatter(np.array(INLET)[:,0],np.array(INLET)[:,1], s = 0.5)
plt.scatter(np.array(FLUID)[:,0],np.array(FLUID)[:,1], s = 0.5)
ax.set_aspect('equal', adjustable='box')

In [6]:
INLET = torch.tensor(INLET, requires_grad=True, dtype=torch.float32).to(device)
FLUID = torch.tensor(FLUID, requires_grad=True, dtype=torch.float32).to(device)
CYLINDER = torch.tensor(CYLINDER, requires_grad=True, dtype=torch.float32).to(device)

In [ ]:
# Train ANN
model = Net().to(device)
model.apply(init_weights)
optimizer_adam = optim.Adam(model.parameters(), lr=1e-3)
optimizer_LBFGS = optim.LBFGS(model.parameters(), lr=1e-3)

if os.path.isfile("loss.csv"):
    os.remove("loss.csv")
    
column_df = pd.DataFrame(columns = ['Epoch','Inlet','Cylinder','Fluid'])
column_df.to_csv('loss.csv', mode = 'a')


EPOCH = 3000
for epoch in tqdm(range(EPOCH)):    
    loss_inlet, loss_cylinder, loss_fluid = loss_fn(INLET, CYLINDER, FLUID)
    
    plot_inlet_loss = loss_inlet
    plot_inlet_loss = plot_inlet_loss.cpu().detach().numpy()
    
    plot_cylinder_loss = loss_cylinder
    plot_cylinder_loss = plot_cylinder_loss.cpu().detach().numpy()
    
    plot_ns_loss = loss_fluid
    plot_ns_loss = plot_ns_loss.cpu().detach().numpy()
    
    # Backward Propagation
    loss = 0
    loss = loss_inlet + loss_cylinder + loss_fluid
    
    if epoch+1 <= 2990:
        optimizer_adam.zero_grad()
        loss.backward()
        optimizer_adam.step()
    else:
        optimizer_LBFGS.step(closure)

    
    if (epoch+1)%1 == 0:
        loss_list = [[epoch+1, plot_inlet_loss, plot_cylinder_loss, plot_ns_loss]]
        loss_df = pd.DataFrame(loss_list)
        loss_df.to_csv('loss.csv', mode = 'a', header = False)
        
    
#     if graph_ns_loss < 1e-4 and epoch > 10:
#         print('Residual tolerance reached')
#         break

print('Training Done!')

In [ ]:
training_loss = pd.read_csv("loss.csv")
epoch_list = training_loss["Epoch"].tolist()
inlet_loss_list = training_loss["Inlet"].tolist()
airfoil_loss_list = training_loss["Cylinder"].tolist()
fluid_loss_list = training_loss["Fluid"].tolist()

fig, ax = plt.subplots(figsize = (15,15))
training_plot_NS = ax.plot(epoch_list,fluid_loss_list, label='NS')
training_plot_Inlet = ax.plot(epoch_list,inlet_loss_list, label='Inlet')
training_plot_Cylinder = ax.plot(epoch_list,airfoil_loss_list, label='Cylinder')
plt.legend()
plt.xlabel('EPOCHS')
plt.ylabel('Error')
plt.yscale('log')
plt.show()

In [ ]:
# Post Processing
x_POST = np.linspace(lb, rb, 250)
y_POST = np.linspace(bb, tb, 250)
x_POST, y_POST = np.meshgrid(x_POST, y_POST)
x_POST = x_POST.flatten()[:, None]
y_POST = y_POST.flatten()[:, None]

POST = np.concatenate((x_POST, y_POST), axis = 1)

bool_POST = []

for point in POST:
    if point[0]**2 + point[1]**2 <= 0.05**2:
        bool_POST.append(False)
    elif point[0] == -0.2:
        bool_POST.append(False)
    else:
        bool_POST.append(True)

COORDS = []
for i in range(len(bool_POST)):
    if bool_POST[i]:
        COORDS.append(POST[i])
COORDS = torch.tensor(COORDS, dtype=torch.float32)
# plt.scatter(COORDS[:,0], COORDS[:,1])

model.to('cpu')
u_POST, v_POST, p_POST = model(COORDS)

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))
cf = ax[0].scatter(COORDS[:,0], COORDS[:,1], c=u_POST.detach().numpy(), alpha=0.5, edgecolors='none', cmap='jet')
fig.colorbar(cf, ax=ax[0])
ax[0].set_aspect('equal', adjustable='box')
ax[0].set_title('u')

cf = ax[1].scatter(COORDS[:,0], COORDS[:,1], c=v_POST.detach().numpy(), alpha=0.5, edgecolors='none', cmap='jet')
fig.colorbar(cf, ax=ax[1])
ax[1].set_aspect('equal', adjustable='box')
ax[1].set_title('v')

cf = ax[2].scatter(COORDS[:,0], COORDS[:,1], c=p_POST.detach().numpy(), alpha=0.5, edgecolors='none', cmap='jet')
fig.colorbar(cf, ax=ax[2])
ax[2].set_aspect('equal', adjustable='box')
ax[2].set_title('p')

In [9]:
torch.save(model.state_dict(), "CYLINDER_10x50_5000_LARGE.pth")